In [1]:

#
# Het Deutsch Josza Algorithm: 
# 1 berekening op een quantum computer kan het oplossen, vergeleken met verschillende stappen in het klassieke geval
#

import qiskit
from qiskit import transpile, ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit_aer import Aer
import numpy as np
#from qiskit import IBMQ
from configparser import RawConfigParser

type = 'sim' # Run program on the simulator or real quantum machine.

def run(program, type, shots = 100):
  if type == 'real':
    if not run.isInit:
        # Setup the API key for the real quantum computer.
        parser = RawConfigParser()
        parser.read('config.ini') #je moet de key via het platform opvragen en in een config;ini file lokaal opslaan.
        IBMQ.enable_account(parser.get('IBM', 'key'))
        run.isInit = True

    # Set the backend server.
    backend = qiskit.providers.ibmq.least_busy(qiskit.IBMQ.backends(simulator=False))

    # uitvoeren op quantum
    print("Running on", backend.name())
    job = qiskit.execute(program, backend)
    return job.result().get_counts()
  else:
    # uitvoeren op simulator
    print("Running on the simulator.")
    simulator = Aer.get_backend('qasm_simulator')
    transpiled_program = transpile(program, simulator)
    job = simulator.run(transpiled_program, shots=shots)
    return job.result().get_counts()

run.isInit = False

# Deutsch-Josza

Initialisatie

In [8]:
# Op hoeveel bits ?
n = 3

# De string die we gaan testen, even random initialiseren
oracleType = np.random.randint(2)
oracleValue = np.random.randint(2)
print(oracleValue)

print("The oracle is constant.") if oracleType == 0 else print("The oracle is balanced.")



1
The oracle is balanced.


Qubits creëren, koppelen aan QuantumCircuit, superpositie maken met alle qubits behalve de laatste; die laatste qubit inverteren en dan andere superpositie maken ( |-> state ipv |+> state voor de laatste.

In [3]:
# Create n + 1 qubits voor de input array, allemaal initieel op 0, met een extra qubit voor het antwoord.
qr = QuantumRegister(n + 1)
# Create n registers voor output.
cr = ClassicalRegister(n)
program = QuantumCircuit(qr, cr)

# Superpositie maken
for i in range(n):
  program.h(qr[i]) # Hadamard gate

# Inverteer de laatste qubit naar 1 (die stelt het antwoord voor) en maak superpositie.
program.x(qr[n])
program.h(qr[n])


In de code hieronder gaan we de 'oracle function' waarvan we willen bepalen of ze constant dan wel gebalanceerd is, definiëren. Dit is dus allemaal 'problem setup'.

In [11]:
# Een barrier dient puur om aan te geven waar bepaalde stukken samenhoren.
# hier duiden we de start aan van het maken van de oracle functie.
program.barrier()

if oracleType == 0:
  # Constant oracle: oracleValue is de return voor elke mogelijke input.
  # oracle is constant, 
  if oracleValue == 1:
    # Inverteer de laatste qubit.
    program.x(qr[n])
  else:
    # doe niks met de laatste qubit.
    program.iden(qr[n]) # dit doet niks, het is de 'identity function'
else:
    # gebalanceerd, dus het oracle geeft de helft van de opties 0, de andere helft 1
  for i in range(n):
    # Set the qubit to return the inner product of the input with a non-zero bitstring.
    
    if (n & (1 << i)): # bit shift operator, vervolgens AND bit operator
      # controlled not tussen input qubit en output subit (de laatste subit)
      program.cx(qr[i], qr[n])

# einde van het oracle process
program.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=4, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(4, 'q1'), 0), Qubit(QuantumRegister(4, 'q1'), 1), Qubit(QuantumRegister(4, 'q1'), 2), Qubit(QuantumRegister(4, 'q1'), 3)), clbits=())

Nu hebben we de qubits dus in de vorm die we willen om het probleem in met 1 meting per qubit op te lossen. dat vergt dus maar n stappen, in plaats van 2^(n-1) + 1 met een gewone computer!

In [5]:
# superpositie ongedaan maken
for i in range(n):
  program.h(qr[i])

# resultaat meten van elke input qubit
program.barrier()
for i in range(n):
  program.measure(qr[i], cr[i])

In [6]:
# The output voor een constant oracle bevat allemaal 0'en, voor gebalanceerde oracles zitten er altijd 1'en tussen. 
# op een echte quantum computer is er ruis, en moet je dus goed kijken naar wat het vaakst voorkomt. Daarom worden experimenten
# typisch vaak herhaald, zoals ook in deze code 100 keer.
# bij gebalanceerde oracle gaat de code qubits outputten waar zeker 1'en tussen zitten.

# verwachte output:

# Simulator - constant oracle.
# {'000': 1024}
# Simulator - gebalanceerd oracle.
# {'011': 1024}
# quantum computer - Tconstant oracle.
# {'110': 2, '000': 879, '100': 75, '010': 29, '101': 5, '011': 1, '001': 33}
# quantum computer - gebalanceerd.
# {'100': 163, '111': 238, '010': 88, '101': 73, '011': 264, '001': 69, '110': 69, '000': 60}
print(run(program, type))

Running on the simulator.
{'000': 100}


Meer details: zie https://qiskit.org/textbook/ch-algorithms/deutsch-jozsa.html